# Clustering

The goal of this program is to visualize and cluster the data.

## Input
* Importing the libaries that are necessary:

In [1]:
import numpy as np
import pandas as pd
import math as mt

* Importing the necessary files:
 * Read files
 * Convert read files to data
 * Save data in dataframes

In [2]:
df_data = pd.read_csv('Data\Voorbeeld_clusterdata.txt', sep='  ', header=None, engine='python', index_col=0)
df_results = pd.read_csv('Data\Voorbeeld_clusterresult.txt', sep='  ', header=None, engine='python', index_col=0)
df_clusterd = df_data.copy().drop(columns=df_data.columns)

df_data

,1,2,3,4,5,6,7,8
0,,,,,,,,
846160,0.388,0.329,0.690,0.900,0.626,0.621,0.399,0.370
820434,-0.296,-0.503,-0.454,-0.868,-0.721,-0.918,-0.486,-0.582
849103,-0.246,-0.935,-0.277,-0.175,-0.278,-0.075,-0.236,-0.417
846353,0.769,0.929,0.977,1.095,1.058,0.864,0.689,0.492
848613,-0.365,-0.760,-0.305,-0.293,-0.364,-0.155,-0.472,-0.606
...,...,...,...,...,...,...,...,...
760857,1.243,1.824,1.569,2.671,1.644,1.305,1.541,1.094
805218,0.810,0.641,0.545,0.973,1.137,0.518,0.711,0.355
945014,-0.486,-0.743,-0.066,0.046,0.364,0.176,0.172,0.006


## Cluster methods
### K-means (KMCA):

uitleggen hoe het werkt

In [3]:
class KMCA:
    def __init__(self, k=6, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        
    def set_k(K):
        self.k = K

    def fit(self,data):

        self.centroids = {}

        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    optimized = False

            if optimized:
                break

    def predict(self,data):
        def pred(self, data):
            distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
            classification = distances.index(min(distances))
            return classification
        
        self.pred_classifications = list()
        
        if np.isfortran(data):
            for element in data:
                self.pred_classifications.append(pred(self, element))
        else:
            self.pred_classifications.append(pred(self, data))
            
        return self.pred_classifications

# Own cluster method = gridbased clustering:

Om grids te maken zal elk coordinaat worden opgesplitst in een j aantal sub-cellen. Vervolgens zal voor elk coordinaat beslist worden in welke sub-cel deze past. Dit zal dus een getal zijn in de range van 1 tot j. Als elke sub-cel is bepaald ontstaat er een unieke code van sub-cellen. Voor elk van codes wordt er gekeken of er aanliggende sub-cellen zijn met datapoints er in. Als dit zo is zullen deze sub-cellen dezelfde groep worden.

## Stap 1:Productie intervallen per coordinaat:
Om de sub-cel groepen te maken, zullen er intervallen worden opgesteld:
1. Ondergrens = minimale waarde van het coordinaat-type
2. Bovengrens = maximale waarde van het coordinaat-type
3. J = het aantal intervallen dat geproduceerd moet worden

In [4]:
def make_grid(data,j=10):
    dic = dict()
    
    for col in data.columns:
        minimum = data[[col]].min().values[0]
        maximum = data[[col]].max().values[0]
        dic[col] = np.mgrid[minimum:maximum:complex(0,j)]
    
    return dic

make_grid(df_data,10)

{1: array([-4.629     , -3.88788889, -3.14677778, -2.40566667, -1.66455556,
        -0.92344444, -0.18233333,  0.55877778,  1.29988889,  2.041     ]),
 2: array([-4.601     , -3.66366667, -2.72633333, -1.789     , -0.85166667,
         0.08566667,  1.023     ,  1.96033333,  2.89766667,  3.835     ]),
 3: array([-5.972     , -4.96944444, -3.96688889, -2.96433333, -1.96177778,
        -0.95922222,  0.04333333,  1.04588889,  2.04844444,  3.051     ]),
 4: array([-11.85 , -10.027,  -8.204,  -6.381,  -4.558,  -2.735,  -0.912,
          0.911,   2.734,   4.557]),
 5: array([-4.255     , -3.41388889, -2.57277778, -1.73166667, -0.89055556,
        -0.04944444,  0.79166667,  1.63277778,  2.47388889,  3.315     ]),
 6: array([-6.208     , -5.23655556, -4.26511111, -3.29366667, -2.32222222,
        -1.35077778, -0.37933333,  0.59211111,  1.56355556,  2.535     ]),
 7: array([-7.447     , -6.32944444, -5.21188889, -4.09433333, -2.97677778,
        -1.85922222, -0.74166667,  0.37588889,  1.49344444

## Stap 2: Productie van cellen:
Om cellen te maken zullen alle sub-cellen van een sample bij elkaar worden gevoegd. Dit gebeurd als volgt:
1. Bepalen in welke sub-cel elk coordinaat van een sample hoort
2. sub-cellen samenvoegen tot 1 lange unieke code. Dit staat gelijk aan het cel-ID

In [5]:
def make_cells(data,j=10):
    df_grid = df_data.copy().drop(columns=df_data.columns)
    cords = make_grid(data,j)

    for cols in cords:
        points = cords[cols]
        df_grid[cols] = ''

        for i in range(1,len(cords[cols])):
            cond = (df_data[cols] >= points[i-1]) & (df_data[cols] < points[i])
            df_grid.loc[df_grid[cond][[cols]].index,cols] = i

    df_grid['cel'] = ''

    for loc in df_grid.index:
        lijst = df_grid[list(df_grid.columns)[:-1]].loc[loc].to_list()
        lst = ''
        for i in lijst:
            lst += str(i)
        df_grid.loc[loc,'cel'] = lst
    
    df_grid = df_grid.drop(columns=cords.keys())
    
    return df_grid

make_cells(df_data,j = 10)

,cel
0,
846160,76776888
820434,65675677
849103,64675777
846353,86787888
848613,65675777
...,...
760857,87888899
805218,86787788
945014,65676777


## Stap 2: Productie van groepen:
Om groepen te maken zullen alle cellen geevalueerd moeten worden:
1. Kijken naar welke cel-ID's er zijn
2. Bepalen van omliggers van elk cel-ID's
3. Bepalen of de potentiele omliggers overeenkomen met bestaande cel-ID's

    3.1 Zo ja: cel-ID's worden samen gegroepeerd
    
    3.2 Zo nee: cel-ID krijgt groep -1

In [6]:
# COMBINE SHIT

def permutations(iterable, r=None):
    # permutations('ABCD', 2) --> AB AC AD BA BC BD CA CB CD DA DB DC
    # permutations(range(3)) --> 012 021 102 120 201 210
    pool = tuple(iterable)
    n = len(pool)
    r = n if r is None else r
    if r > n:
        return
    indices = list(range(n))
    cycles = list(range(n, n-r, -1))
    yield tuple(pool[i] for i in indices[:r])
    while n:
        for i in reversed(range(r)):
            cycles[i] -= 1
            if cycles[i] == 0:
                indices[i:] = indices[i+1:] + indices[i:i+1]
                cycles[i] = n - i
            else:
                j = cycles[i]
                indices[i], indices[-j] = indices[-j], indices[i]
                yield tuple(pool[i] for i in indices[:r])
                break
        else:
            return

## Stap 2.1: Neigbours bepalen van cel_0
Hier zijn verschillende mogelijkheden denkbaar. Namemlijk:

### Optie 1: Cross

In [7]:
def cross(data):
    def add(data_n,n):
        solutions = list()
        
        for i in [-1,1]:
            data = data_n.copy()
            data[n] += i
            
            solutions.append(data)
        
        return(tuple(solutions))
             
    options = list()    
    
    for n in range(len(data)):
        options.append(add(data,n))
        
    return options

data = [7, 7, 7, 8, 7, 8, 8, 8]
cross(data)

[([6, 7, 7, 8, 7, 8, 8, 8], [8, 7, 7, 8, 7, 8, 8, 8]),
 ([7, 6, 7, 8, 7, 8, 8, 8], [7, 8, 7, 8, 7, 8, 8, 8]),
 ([7, 7, 6, 8, 7, 8, 8, 8], [7, 7, 8, 8, 7, 8, 8, 8]),
 ([7, 7, 7, 7, 7, 8, 8, 8], [7, 7, 7, 9, 7, 8, 8, 8]),
 ([7, 7, 7, 8, 6, 8, 8, 8], [7, 7, 7, 8, 8, 8, 8, 8]),
 ([7, 7, 7, 8, 7, 7, 8, 8], [7, 7, 7, 8, 7, 9, 8, 8]),
 ([7, 7, 7, 8, 7, 8, 7, 8], [7, 7, 7, 8, 7, 8, 9, 8]),
 ([7, 7, 7, 8, 7, 8, 8, 7], [7, 7, 7, 8, 7, 8, 8, 9])]

### Optie 2: Rectangle

In [8]:
def rectangle(data):
    def add(data_n,n):
        options = list()
        
        for i in range(-1,2):
            data = data_n.copy()
            data[n-1] += i
            
            options.append(data)
        
        result = list()
        
        for option in options:
            result.append(option)
            
            for j in [-1,1]:
                res = option.copy()
                res[n] += j

                result.append(res)
                
        return tuple(result)
    
    
    options = list()    
    
    for n in range(1,len(data)):
        options.append(add(data,n))
        
    return options

data = [7, 7, 7, 8, 7, 8, 8, 8]
rectangle(data)

[([6, 7, 7, 8, 7, 8, 8, 8],
  [6, 6, 7, 8, 7, 8, 8, 8],
  [6, 8, 7, 8, 7, 8, 8, 8],
  [7, 7, 7, 8, 7, 8, 8, 8],
  [7, 6, 7, 8, 7, 8, 8, 8],
  [7, 8, 7, 8, 7, 8, 8, 8],
  [8, 7, 7, 8, 7, 8, 8, 8],
  [8, 6, 7, 8, 7, 8, 8, 8],
  [8, 8, 7, 8, 7, 8, 8, 8]),
 ([7, 6, 7, 8, 7, 8, 8, 8],
  [7, 6, 6, 8, 7, 8, 8, 8],
  [7, 6, 8, 8, 7, 8, 8, 8],
  [7, 7, 7, 8, 7, 8, 8, 8],
  [7, 7, 6, 8, 7, 8, 8, 8],
  [7, 7, 8, 8, 7, 8, 8, 8],
  [7, 8, 7, 8, 7, 8, 8, 8],
  [7, 8, 6, 8, 7, 8, 8, 8],
  [7, 8, 8, 8, 7, 8, 8, 8]),
 ([7, 7, 6, 8, 7, 8, 8, 8],
  [7, 7, 6, 7, 7, 8, 8, 8],
  [7, 7, 6, 9, 7, 8, 8, 8],
  [7, 7, 7, 8, 7, 8, 8, 8],
  [7, 7, 7, 7, 7, 8, 8, 8],
  [7, 7, 7, 9, 7, 8, 8, 8],
  [7, 7, 8, 8, 7, 8, 8, 8],
  [7, 7, 8, 7, 7, 8, 8, 8],
  [7, 7, 8, 9, 7, 8, 8, 8]),
 ([7, 7, 7, 7, 7, 8, 8, 8],
  [7, 7, 7, 7, 6, 8, 8, 8],
  [7, 7, 7, 7, 8, 8, 8, 8],
  [7, 7, 7, 8, 7, 8, 8, 8],
  [7, 7, 7, 8, 6, 8, 8, 8],
  [7, 7, 7, 8, 8, 8, 8, 8],
  [7, 7, 7, 9, 7, 8, 8, 8],
  [7, 7, 7, 9, 6, 8, 8, 8],
  [7, 7, 7, 9, 8,

### Optie 3: Cubus
Hier zijn r^n mogelijkheden voor.
* r staat voor de range aan cellen naast cel_0
* n staat voor het aantal dimensies

In ons geval is dit: 3^8 = 6561 mogelijheden ==> 6560 mogelijkheden zonder cel_0 mee te rekenen

Elk van de aanliggende cellen worden achterhaald doormiddel van een algemene liberary op te stellen met alle mogelijke locaties genormaliseerd. Om vervolgens de daadwerkelijke aanliggende cellen van cel-ID's te bepalen kan de liberary worden toegepast aan het het cell-ID van cell_0.

In [9]:
def sub(length):
    list_0 = [0 for i in range(length)]

    def make_list(data,n):
        test = data.copy()
        var = list()
        
        for i in range(len(list_0)):
            test = test.copy()
            test[i] = n
            var.append(test)
            
        return var
    
    lists_neg = make_list(list_0,-1)
    lists_pos = make_list(list_0,1)
    lists_mix = make_list(lists_pos[-1],-1)
    lists_mix.reverse()
    
    lists_mix0 = list()
    for posib in lists_mix[1:-1]:
        lists_mix0.append(make_list(posib,0))
    
    lists_mix_0 = list()
    for j in range(len(lists_mix0)):
        for k in range(len(lists_mix0[j])):
            if 1 and -1 in lists_mix0[j][k]:
                lists_mix_0.append(lists_mix0[j][k])
                
    del lists_mix0
    
    joined = lists_neg[:-1] + lists_pos[:-1] + lists_mix[1:] + lists_mix_0
    
    return tuple((list_0,lists_neg[-1],lists_pos[-1])), joined

def posibilaties(length):
    extremes, data = sub(length)
    
    lib = {}
    for i in range(len(extremes)):
        lib[i] = extremes[i]
    
    uniques = list()
    for j in range(len(data)):
        uniques.append(np.unique([i for i in permutations(data[j])], axis=0))
    
    num = i
    for unique in uniques:
        for values in unique:
            num += 1
            lib[num] = values
            
    return lib
    
lib_pos = posibilaties(8)

In [10]:
len(lib_pos)

6561

## Stap 2.2: cross refressing:

In [11]:
tresh = round(df_grid['cel'].value_counts().quantile(q=0.25))

for cordinates in df_cluster.values:
    for i in lib_pos:
        neigb = np.add(cordinates,lib_pos[i])

        if neigb in df_grid.#add col of lists:
            if neigb.value_counts() >= tresh:
                df_grid['group'] = #group
            else:
                df_grid['group'] = -1
        else:
            df_grid['group'] = -2    

SyntaxError: invalid syntax (<ipython-input-11-48a0aedf9292>, line 9)

## Output

* get cluster labels
* save labels in text file

### get cluster labels:

In [ ]:
kmca = KMCA()
kmca.fit(df_data.values)

df_clusterd['KMCA'] = kmca.predict(df_data.values)
#df_clusterd['GCA'] = 

df_clusterd.head()

### save results:

In [ ]:
#df_clusterd.to_csv(r'Data\clusterresultaten.txt', header=None, index=True, sep=' ')